In [18]:
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astroquery.utils.tap.core import TapPlus
import importlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
sys.path.append('../../')

from astroLuSt.database_interface import (
    database_utils as aldidu,
    simbadinterface as aldisi
)
from astroLuSt.styles import plot_styles as alstps

importlib.reload(aldidu)

#style for plotting
_ = alstps.tre_light()



# `get_reference_objects()`
* searching fore closeby constant reference objects

In [3]:
#function to filter table of results
def filter_func(
    tab:Table
    ) -> Table:
    """
        - function to get relevant target out of nearby reference targets
    """
    
    #reference object with distance closest to the median of all distances
    filtered = tab[np.argsort(np.abs(tab['distance']-np.median(tab['distance'])))][-1:]
    
    # #reference object closest to science target
    # filtered = tab[np.argsort(tab['distance'])][-1:]
    
    # #reference object furthest from science target
    # filtered = tab[np.argsort(tab['distance'])][:1]
    
    # #all reference objects
    # filtered = tab

    return filtered

In [4]:
importlib.reload(aldidu)

#get coordinates
SDI = aldisi.SimbadDatabaseInterface(n_jobs=1)
df_ids = SDI.get_ids(["RR Lyr", "TV Boo"])
coords = SkyCoord(df_ids["ra"], df_ids["dec"], unit="deg")

#extract potential reference ojects
res_tab1 = aldidu.get_reference_objects(coords=coords[0], radius=.05)
res_tab2 = aldidu.get_reference_objects(coords=coords[1], radius=.1)

display(res_tab1)
display(res_tab2)

#filter for relevant objects
res_tab1 = filter_func(res_tab1)
res_tab2 = filter_func(res_tab2)


oid,ra,dec,main_id,otype,ids,distance,vartyp_mesvar
,deg,deg,,,,,
int64,float64,float64,object,object,object,float64,object
13657430,291.40124541359,42.80931251755,Gaia DR3 2125984008083980928,*,Gaia DR3 2125984008083980928|Gaia DR2 2125984008083980928,0.03577733430113383,
12448530,291.41752353239997,42.76032540198,2MASS J19254020+4245371,*,Gaia DR3 2125983046017836544|TIC 159717459|2MASS J19254020+4245371|KIC 7199152|Gaia DR2 2125983046017836544,0.04462333254031259,


oid,ra,dec,main_id,otype,ids,distance,vartyp_mesvar
,deg,deg,,,,,
int64,float64,float64,object,object,object,float64,object
21037280,214.15861818394293,42.26231737542833,Gaia DR3 1492228976169235200,*,Gaia DR3 1492228976169235200|Gaia DR2 1492228976169235200,0.09771036592599006,


/home/lukas/venvs/astroLuSt/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:870: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


# `query_upload_table()`
* uploading a table in pieces to execute larger query

In [17]:
importlib.reload(aldidu)

df = pd.DataFrame(data={
    'id':[
        4295806720, 38655544960, 343597448960,
        549755818112, 828929527040, 1275606125952,
        1340029955712, 1374389600384, 1619203481984,
        1653563247744, 2199023259904, 2546916445184,
        2851858288640, 2920577765120, 2989297243776,
        3062312520192, 3332894779520, 3371550165888,
    ]
}, dtype=int)

# query = f"""
#     SELECT
#         ut.*, sh.Source
#     FROM tap_upload.upload_table AS ut INNER JOIN
#         "I/354/starhorse2021" AS sh
#             ON CAST(ut.id AS BIGINT) = sh.Source AS BIGINT
# """
# tap = TapPlus(
#     url='http://tapvizier.cds.unistra.fr/TAPVizieR/tap',
#     tap_context=None,
#     verbose=True,
# )

query = f"""
    SELECT
        ut.*, gaiadr3.source_id
    FROM tap_upload.upload_table AS ut INNER JOIN
        gaiadr3.gaia_source_lite AS gaiadr3
            ON ut.id = gaiadr3.source_id
"""
tap = TapPlus(
    url="https://gea.esac.esa.int/tap-server/tap",
    tap_context=None,
    verbose=True,
)

df_res = aldidu.query_upload_table(
    tap=tap, query=query,
    df_upload=df,
    upload_table_name='upload_table',
    query_async=False,  #no asynchronous query needed, as table is small
    nsplits=3,
    verbose=2,
    parallel_kwargs=dict(n_jobs=3, verbose=3),
    launch_job_kwargs=dict(verbose=False),
)

df_res

Created TAP+ (v20200428.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
INFO(query_upload_table): Extracting split 1/3 (len(split): 6)
INFO(query_upload_table): Extracting split 2/3 (len(split): 6)
INFO(query_upload_table): Extracting split 3/3 (len(split): 6)


/home/lukas/venvs/astroLuSt/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.5s finished


,id,SOURCE_ID
0,4295806720,4295806720
1,38655544960,38655544960
2,343597448960,343597448960
3,549755818112,549755818112
4,828929527040,828929527040
5,1275606125952,1275606125952
0,1340029955712,1340029955712
1,1374389600384,1374389600384
2,1619203481984,1619203481984
3,1653563247744,1653563247744
